In [1]:
import json
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import sys
sys.path.insert(0, '../utils')
import data_utils

sys.path.insert(0, '../models')
import log_reg_word_embeddings

In [3]:
df_train = data_utils.load_data_set('../data/train/train.csv')
df_train_x = df_train[['head.word', 'tail.word', 'sentence']]
df_train_y = df_train['relation']

In [4]:
model = log_reg_word_embeddings.LogRegWordEmbeddings()

In [5]:
df_train_features = model.transform(df_train_x)

Tokenizing sentences...
0    [she, also, oversaw, refinancing, state, super...
1    [more, 2, 5, million, cubic, yards, contaminat...
2    [the, onondaga, nation, 1, 500, members, feder...
3    [he, born, istanbul, raised, eastern, city, ad...
4    [by, end, recent, tour, rollins, met, soldiers...
Name: sentence, dtype: object
Averaging word embeddings...
0    [-0.024940285714285717, -0.014328357142857137,...
1    [0.022366650000000002, 0.060093550000000016, -...
2    [-0.02160072727272727, 0.10642886363636364, -0...
3    [0.012861874999999998, 0.0178651875, -0.102017...
4    [0.07083508108108107, 0.03429608108108108, -0....
Name: sentence, dtype: object


In [6]:
# model.fit(df_train_x, df_train_y)